In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install darts

In [ ]:
from darts.models.forecasting.dlinear import DLinearModel
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [ ]:
processed_data_dir = '/content/drive/MyDrive/M5-A1/2. data/processed/'
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
ver, KKK = 'priv', 0
START_TRAIN = 41
END_TRAIN   = 1941 - 28*KKK
P_HORIZON   = 28
USE_AUX     = False
TARGET = 'sales'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std']

In [ ]:
processed_data_dir_base = '/content/drive/MyDrive/M5-thesis/M5-FITS/processed/'
global_df = pd.read_pickle(processed_data_dir_base + 'raw_data.pkl')

In [ ]:
from darts.timeseries import TimeSeries

TRAIN_START = 0
TEST_START = 1941
HORIZON = 28
VER = 1.0
INPUT_MULTIPLIER = 0.5 # 1.0 for full
MODEL_NAME='100-epoch-no-params-0.5-input'
stores = global_df['store_id'].unique()
depts = global_df['dept_id'].unique()
stores, depts

(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
 Categories (10, object): ['CA_1', 'CA_2', 'CA_3', 'CA_4', ..., 'TX_3', 'WI_1', 'WI_2', 'WI_3'],
 ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']
 Categories (7, object): ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1',
                          'HOUSEHOLD_2'])

In [ ]:
"""
get df_train and features
"""
def get_data_by_store_id(store_id, dept_id):
  df = global_df.copy()

  store_data = df[(df['store_id'] == store_id) & (df['dept_id'] == dept_id) & (df['d'] >= TRAIN_START)]
  df_train = store_data.pivot(index='id', columns='d', values='sales')
  df_train.fillna(0, inplace=True)
  df_calendar = store_data.groupby(['d']).agg({
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
  }).reset_index()
  df_calendar.index = df_calendar['d']

  del df
  gc.collect()

  return df_train, df_calendar, store_data

In [ ]:
df_train, df_calendar, store_data = get_data_by_store_id('CA_1', 'HOBBIES_1')
df_train

d,1,2,3,4,5,6,7,8,9,10,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_008_CA_1_evaluation,12.0,15.0,0.0,0.0,0.0,4.0,6.0,5.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_009_CA_1_evaluation,2.0,0.0,7.0,3.0,0.0,2.0,3.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_010_CA_1_evaluation,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_012_CA_1_evaluation,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_015_CA_1_evaluation,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_1_125_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_232_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_269_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import math

class M5DLinear(object):
  def __init__(self, horizon, df_train, df_calendar, epochs, lookback, kernel_size=7):
    self.horizon = horizon
    self.df_train = df_train
    self.df_calendar = df_calendar
    self.exog_fit, self.exog_pred = self.get_exog_variables()
    self.top_level = self.get_top_level()
    self.train_len = len(self.df_train.columns)
    self.epochs = epochs
    self.lookback = lookback
    self.kernel_size = kernel_size

  def get_top_level(self):
    """
    Calculate top level time series by aggregating low level time-series
    """
    data = np.sum(self.df_train.values, axis=0)

    return data

  def get_exog_variables(self):
    self.df_calendar['snap_CA'] = self.df_calendar['snap_CA'].astype(int)
    self.df_calendar['snap_TX'] = self.df_calendar['snap_TX'].astype(int)
    self.df_calendar['snap_WI'] = self.df_calendar['snap_WI'].astype(int)
    calendar = self.df_calendar[['d', 'snap_CA','snap_TX','snap_WI', 'event_name_1', 'event_name_2', 'tm_w_end']].copy()
    calendar['snap_count'] = calendar[['snap_CA','snap_TX','snap_WI']].apply(np.sum,axis=1)

    calendar['is_event_1'] = [isinstance(x , str)*1 for x in calendar['event_name_1']]
    calendar['is_event_2'] = [isinstance(x , str)*1 for x in calendar['event_name_2']]
    calendar['is_event'] = calendar[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
    calendar['is_event'] = np.where(calendar['is_event']>0,1,0)

    exog_fit = calendar[['snap_count', 'is_event', 'tm_w_end']].iloc[:TEST_START,:]
    exog_fit.index = range(len(exog_fit))
    exog_pred = calendar[['snap_count', 'is_event', 'tm_w_end']].iloc[TRAIN_START:TEST_START+self.horizon,:]
    exog_pred.index = range(len(exog_pred))

    exog_fit = TimeSeries.from_dataframe(exog_fit, value_cols=exog_fit.columns)
    exog_pred = TimeSeries.from_dataframe(exog_pred, value_cols=exog_fit.columns)

    return exog_fit, exog_pred

  def get_weights(self):
    w = np.sum(self.df_train.iloc[:, -28:].values, axis=1) / sum(self.top_level[-28:])
    w = w.reshape(len(w), 1)

    return w

  def predict_top_level(self):
    print(f"training with {self.epochs} epochs and {self.lookback} lb window")
    self.model = DLinearModel(
      input_chunk_length=self.lookback,
      output_chunk_length=self.horizon,
      n_epochs=self.epochs,
      model_name='dlinear_model',
      random_state=42,
      kernel_size=self.kernel_size
    )

    y_train = TimeSeries.from_values(self.get_top_level())

    self.model.fit(
        y_train,
        past_covariates=self.exog_fit,
        future_covariates=self.exog_pred,
        verbose=False
    )
    preds = self.model.predict(
        self.horizon,
        past_covariates=self.exog_fit,
        future_covariates=self.exog_pred
    )

    return preds.values()

  def predict_bottom_level(self):
    w = self.get_weights()

    top_level_preds = self.predict_top_level()
    top_level_preds = top_level_preds.reshape(1,len(top_level_preds))
    preds = np.multiply(top_level_preds, w)
    item_ids = self.df_train.index
    res = pd.DataFrame({
        'id': item_ids
    })

    predictions_df = pd.DataFrame(preds, columns=[f'F{i+1}' for i in range(28)])
    preds = pd.concat([res['id'], predictions_df], axis=1)

    preds.set_index('id', inplace=True)

    return preds

In [ ]:
def get_preds(df_train, df_calendar, epochs, lookback, kernel_size):
  model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:1941].copy(), df_calendar=df_calendar, epochs=epochs, lookback=lookback, kernel_size=kernel_size)
  preds = model.predict_bottom_level()

  return preds

def train_model(lookback, epochs, kernel_size, model_name):
  submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
  submission.set_index('id', inplace=True)
  for store_id in stores:
    for dept_id in depts:
      df_train, df_calendar, store_data = get_data_by_store_id(store_id, dept_id)
      preds = get_preds(df_train, df_calendar, epochs, lookback, kernel_size)

      submission.update(preds)

      del df_train
      del df_calendar
      gc.collect()

  submission.reset_index(inplace=True)
  submission.to_csv(f"/content/drive/MyDrive/M5-thesis/submissions/{model_name}.csv", index=False)

In [ ]:
for kernel_size in [7, 14, 21, 28]:
  model_name = f"dlinear-lb-{56}-ep-{100}-ks-{kernel_size}-{VER}"
  train_model(56, 100, kernel_size, model_name)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


training with 100 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


training with 100 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
df_train, df_calendar = get_data_by_store_id('TX_1', 'FOODS_3')
model = M5DLinear(horizon=28, df_train=df_train.iloc[:, 0:1941].copy(), df_calendar=df_calendar, epochs=100, lookback=56)
preds = model.predict_bottom_level()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


training with 100 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
##### EVAL
def get_true_values(store_id, dept_id):
  true_df = pd.read_csv('/content/drive/MyDrive/data/m5-data/sales_test_evaluation.csv')
  true_df = true_df[(true_df['store_id'] == store_id) & (true_df['dept_id'] == dept_id)]
  true_df['id'] = true_df['item_id'] + '_' + true_df['store_id'] + '_evaluation'
  true_df.set_index('id', inplace=True)

  return true_df

def get_weights(raw_df, h=28):
  length = len(raw_df['d'].unique())
  temp = raw_df[raw_df['d'] >= length - 2 * h]
  temp['sell_price'] = temp['sell_price'].astype('float64')

  sale_df = temp.pivot(index='id', columns='d', values='sales')
  prices_df = temp.pivot(index='id', columns='d', values='sell_price')
  sale_df.fillna(0, inplace=True)
  prices_df.fillna(0, inplace=True)

  sales = sale_df.iloc[:, -2*h:-h]
  prices = prices_df.iloc[:, -2*h:-h]
  revenue_lm = (sales * prices).sum(axis=1)
  revenue_pct = revenue_lm / revenue_lm.sum()

  return revenue_pct

def df_wide(df):
  temp = df.pivot(index='id', columns='d', values='sales')
  temp.fillna(0, inplace=True)

  return temp

def calculate_rmsse(train, test, forecast):
  forecast_mse = np.mean((test - forecast)**2, axis=1)
  train_mse = [np.mean((np.diff(np.trim_zeros(row))**2)) for row in train]

  return np.sqrt(forecast_mse / train_mse)

In [ ]:
df_train, df_calendar, store_data = get_data_by_store_id('CA_1', 'HOBBIES_1')


In [ ]:
#rmsse
wide = df_wide(store_data).iloc[:, :-28]
preds = get_preds(df_train, df_calendar, 100, 56)
true = get_true_values('CA_1', 'HOBBIES_1')
rmsse = calculate_rmsse(wide.values, true.iloc[:, -28:].values, preds.iloc[:, -28:].values)
w = get_weights(store_data)
wrmsse = w * rmsse
wrmsse

training with 100 epochs and 56 lb window


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,0
id,
HOBBIES_1_001_CA_1_evaluation,0.003561
HOBBIES_1_002_CA_1_evaluation,0.000097
HOBBIES_1_003_CA_1_evaluation,0.000590
HOBBIES_1_004_CA_1_evaluation,0.012035
HOBBIES_1_005_CA_1_evaluation,0.000327
...,...
HOBBIES_1_420_CA_1_evaluation,0.001916
HOBBIES_1_421_CA_1_evaluation,0.000925
HOBBIES_1_422_CA_1_evaluation,0.000038


In [ ]:
wrmsse.sum()

2.7474033361535177

,item_id,dept_id,cat_id,store_id,state_id,d_1942,d_1943,d_1944,d_1945,d_1946,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,0,0,...,2,1,2,0,0,1,0,1,3,1
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2,0,1,0,...,1,0,0,1,0,0,2,1,1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,...,1,3,2,1,0,2,1,0,1,1
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,6,...,3,3,4,2,1,6,3,1,4,3
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2,0,1,1,2,...,0,1,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,1,1,0,...,1,4,0,0,1,1,1,1,1,0
30486,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,1,1,0,...,4,1,0,1,0,1,0,0,0,0
30487,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1,0,0,0,0,...,1,1,2,1,2,1,1,1,1,0
30488,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,1,...,0,0,2,0,1,3,0,2,1,5


In [ ]:
models = [
    {
        'window': 7,
        'name': 'DLinear 7LB'
    },
    {
        'window': 14,
        'name': 'DLinear 14LB'
    },
    {
        'window': 21,
        'name': 'DLinear 28LB'
    },
    {
        'window': 28,
        'name': 'DLinear 28LB'
    },
    {
        'window': 56,
        'name': 'DLinear 56LB'
    },
    {
        'window': 112,
        'name': 'DLinear 7LB'
    },
]

In [ ]:
!pip install torchinfo

In [ ]:
res = []
df, cal, store = get_data_by_store_id('CA_1', 'FOODS_1')

for model in models:
  dl = M5DLinear(28, df.iloc[:, 0:1941], df_calendar, 50, model['window'])
  dl.predict_top_level()
  trainable_params = sum(p.numel() for p in dl.model.model.parameters() if p.requires_grad)
  res.append({"model": f"DLinear-{model['window']}", "params": trainable_params})

In [ ]:
res = pd.DataFrame(res, columns=['model', 'params'])
res

,model,params
0,DLinear-7,2804
1,DLinear-14,5548
2,DLinear-21,8292
3,DLinear-28,11036
4,DLinear-56,22012
5,DLinear-112,43964


In [ ]:
trainable_params = sum(p.numel() for p in model.model.model.parameters() if p.requires_grad)
trainable_params

11036